# Mental State Detection - Exploratory Notebook

EDA and feature extraction for **Mental State Detection from Speech**.
- MFCC, spectral, pitch, ZCR, RMS features
- Data augmentation hooks included
- Ready for further training notebooks

Notes:
- Adjust `DATA_PATH` to point to your `.wav` dataset organized as `DATA_PATH/<label>/*.wav`.
- This notebook focuses on feature extraction and exploratory analysis.

In [ ]:
# 1) Imports
import os, random, warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import librosa, librosa.display
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12,7)
warnings.filterwarnings('ignore')
np.random.seed(42)
random.seed(42)

## 2) Parameters & Paths

In [ ]:
DATA_PATH = "Bale_records"   # path to dataset
AUGMENT_FACTOR = 2  # how many augmented copies per file
SR = 16000
N_MFCC = 13

## 3) Feature Extraction Helpers

In [ ]:
def extract_features_file(file_path, target_sr=SR, n_mfcc=N_MFCC):
    try:
        y, sr = librosa.load(file_path, sr=target_sr)
        if y is None or y.size == 0:
            return None
        y = librosa.util.normalize(y)
        feats = []
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        feats.extend(np.mean(mfcc, axis=1).tolist())
        feats.extend(np.std(mfcc, axis=1).tolist())
        feats.append(np.mean(librosa.feature.spectral_centroid(y=y, sr=sr)))
        feats.append(np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr)))
        feats.append(np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr)))
        feats.append(np.mean(librosa.feature.zero_crossing_rate(y)))
        feats.append(np.mean(librosa.feature.rms(y=y)))
        pitches, mags = librosa.piptrack(y=y, sr=sr)
        mag_mask = mags > np.median(mags[mags > 0]) if np.any(mags>0) else mags > 0
        pitch_vals = pitches[mag_mask]
        feats.append(np.median(pitch_vals) if pitch_vals.size > 0 else 0.0)
        return np.array(feats, dtype=float)
    except Exception:
        return None

def load_dataset_features(data_path=DATA_PATH):
    X_list, y_list = [], []
    for root, dirs, files in os.walk(data_path):
        if root == data_path: continue
        label = os.path.basename(root)
        for f in files:
            if not f.lower().endswith('.wav'): continue
            fp = os.path.join(root, f)
            feats = extract_features_file(fp)
            if feats is None: continue
            X_list.append(feats)
            y_list.append(label)
    X = np.vstack(X_list)
    y = np.array(y_list)
    return X, y

## 4) Load Dataset & Explore

In [ ]:
X, y_labels = load_dataset_features(DATA_PATH)
print('Samples:', X.shape[0], 'Features:', X.shape[1])
print('Classes:', np.unique(y_labels))

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Encode labels
le = LabelEncoder()
y_enc = le.fit_transform(y_labels)
classes = le.classes_
print('Encoded classes:', classes)

## 5) Exploratory Data Analysis

In [ ]:
# Class distribution
plt.figure(figsize=(12,5))
sns.countplot(x=y_labels, order=classes)
plt.title('Class distribution')
plt.show()

# PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)
plt.figure(figsize=(10,7))
for i, cls in enumerate(classes):
    plt.scatter(X_pca[y_enc==i,0], X_pca[y_enc==i,1], label=cls, alpha=0.7)
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.title('PCA of Audio Features')
plt.legend()
plt.show()